In [105]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [106]:
import requests

In [107]:
spark = SparkSession.builder \
    .appName("Apex Financial Group")\
    .getOrCreate()

In [108]:
carteira_investimentos = spark.read.csv("/home/wells/Documentos/Apex Financial Group/dados/carteira_investimentos.csv", header=True, inferSchema=True)

In [109]:
historico_transacoes = spark.read.csv("/home/wells/Documentos/Apex Financial Group/dados/historico_transacoes.csv", header=True, inferSchema=True)

In [110]:
registo_cllientes = spark.read.csv("/home/wells/Documentos/Apex Financial Group/dados/registo_clientes.csv", header=True, inferSchema=True)

In [111]:
solicitacoes_emprestimos = spark.read.csv("/home/wells/Documentos/Apex Financial Group/dados/solicitacoes_emprestimo.csv", header=True, inferSchema=True)

Relação cliente × empréstimos:

Para cada cliente, conte o número de solicitações de empréstimo e o valor total solicitado.


In [118]:
carteira_investimentos.createOrReplaceTempView("carteira")
historico_transacoes.createOrReplaceTempView("transacoes")
registo_cllientes.createOrReplaceTempView("clientes")
solicitacoes_emprestimos.createOrReplaceTempView("solicitacoes")

In [113]:
solicitacoes_emprestimos.show()

+--------------+----------+----------------+--------------------+-----------------+-----------+----------+--------+-------+---------------------+---------------------+
|id_solicitacao|id_cliente|data_solicitacao|valor_solicitado_brl|taxa_juros_mensal|prazo_meses|finalizade|aprovado|default|dias_atraso_historico|score_risco_calculado|
+--------------+----------+----------------+--------------------+-----------------+-----------+----------+--------+-------+---------------------+---------------------+
|             1|       493|      2023-02-07|            20874.01|             0.77|         24| Concluída|   false|  false|                   89|               776.34|
|             2|       128|      2025-08-01|            12753.96|              4.3|         36|  Pendente|    true|  false|                   18|               617.58|
|             3|       138|      2024-03-15|             4238.06|             4.08|          6|  Pendente|    true|  false|                   65|               

In [114]:
spark.sql("""
    SELECT c.id_cliente, count(distinct s.id_solicitacao) as total_solicitacoes
    FROM clientes c
    LEFT JOIN solicitacoes s ON c.id_cliente = s.id_cliente
    GROUP BY c.id_cliente
    ORDER BY total_solicitacoes DESC
""").show()

+----------+------------------+
|id_cliente|total_solicitacoes|
+----------+------------------+
|       465|                 6|
|        22|                 5|
|        41|                 5|
|        12|                 4|
|        47|                 4|
|        13|                 4|
|        92|                 4|
|       257|                 4|
|       395|                 4|
|        29|                 4|
|       170|                 4|
|        68|                 4|
|       407|                 4|
|       393|                 4|
|        61|                 3|
|       255|                 3|
|       202|                 3|
|       291|                 3|
|       310|                 3|
|         3|                 3|
+----------+------------------+
only showing top 20 rows


listagem detalhada dos clientes que possuem registros em todas as quatro tabelas analisadas (clientes, solicitacoes, carteira e transacoes)

In [115]:
spark.sql("""
    SELECT 
        c.id_cliente, 
        c.nome_completo, 
        ct.quantidade, 
        ct.custo_medio_da_compra, 
        h.dispositivo
    FROM clientes c
    INNER JOIN solicitacoes s ON c.id_cliente = s.id_cliente
    INNER JOIN carteira ct ON c.id_cliente = ct.id_cliente
    INNER JOIN transacoes h ON c.id_cliente = h.id_cliente
    ORDER BY ct.quantidade DESC
""").show()

+----------+--------------------+----------+---------------------+-----------+
|id_cliente|       nome_completo|quantidade|custo_medio_da_compra|dispositivo|
+----------+--------------------+----------+---------------------+-----------+
|       277|         Renan Sales|    999.91|                38.25|        web|
|       277|         Renan Sales|    999.91|                38.25|        web|
|        20|  Srta. Ísis Andrade|    995.97|                16.03|        web|
|       117|  José Pedro Rezende|    985.37|                86.26|    celular|
|        53|Dr. Vitor Hugo An...|    982.56|                13.01|        web|
|        53|Dr. Vitor Hugo An...|    982.56|                13.01|        web|
|         2|     Hellena Machado|    978.71|                48.99|        web|
|         2|     Hellena Machado|    978.71|                48.99|    celular|
|         2|     Hellena Machado|    978.71|                48.99|    celular|
|         2|     Hellena Machado|    978.71|        

MOSTRAR CADA AÇÃO QUE CADA CLIENTE POSSUI

In [116]:
spark.sql("""
    SELECT
        c.id_cliente, 
        c.nome_completo, 
        ct.ticker_ativo,
        ct.quantidade,
        ct.custo_medio_da_compra
        
    FROM 
        clientes c
    INNER JOIN carteira ct 
    ON c.id_cliente = ct.id_cliente
    GROUP BY c.id_cliente, c.nome_completo, ct.ticker_ativo, ct.quantidade, ct.custo_medio_da_compra
    ORDER BY c.nome_completo ASC;
          """).show()

+----------+--------------------+------------+----------+---------------------+
|id_cliente|       nome_completo|ticker_ativo|quantidade|custo_medio_da_compra|
+----------+--------------------+------------+----------+---------------------+
|       182|       Agatha Farias|       BBDC4|    673.47|                71.67|
|        24|      Agatha Moreira|       MGLU3|    798.18|                32.87|
|       213|Alexandre Albuque...|       ABEV3|    223.41|                53.67|
|       213|Alexandre Albuque...|       VALE3|    835.36|                63.49|
|       294|      Alexia Azevedo|       BBDC4|     539.8|                 73.7|
|       337|      Alexia Azevedo|       RENT3|     232.0|                15.28|
|       294|      Alexia Azevedo|       BBAS3|     712.4|                44.68|
|       200|         Alexia Melo|       MGLU3|    672.18|                80.82|
|       161|      Alexia Pimenta|       ABEV3|    380.56|                 19.5|
|       161|      Alexia Pimenta|       

FILTRAR CLIENTES COM SCORE ALTO, ACIMA DE 400

In [126]:
spark.sql("""
    SELECT 
          id_cliente,
          nome_completo,
          FLOOR(months_between(current_date(), data_nascimento) / 12) AS idade,
          score_credito_inicial
    FROM 
         clientes
    WHERE 
          score_credito_inicial > 400.00
    ORDER BY 
          score_credito_inicial DESC
""").show()


+----------+--------------------+-----+---------------------+
|id_cliente|       nome_completo|idade|score_credito_inicial|
+----------+--------------------+-----+---------------------+
|       306|    Sra. Laura Pires|   73|               849.25|
|        44|Ana Sophia Rodrigues|   35|               847.48|
|       399|Sra. Camila Nasci...|   43|               846.42|
|       155|      Aylla Carvalho|   19|               845.74|
|       290|   Caroline Teixeira|   60|               844.02|
|       439|          Ana Vieira|   18|               843.66|
|        88|    Vinicius Fonseca|   51|               843.34|
|       246|        Diogo Araújo|   66|               842.59|
|        43|Srta. Giovanna Pires|   39|               841.92|
|        38|     Anthony Moreira|   24|               841.29|
|       368|       Aurora Moraes|   71|               840.65|
|       415|             Ayla Sá|   48|               839.73|
|        65|  Ana Clara Siqueira|   49|               836.91|
|       

DETECÇÃO DE CLIENTES INATIVOS

In [138]:
spark.sql("""
          select
              c.id_cliente,
              c.nome_completo,
              C.status_conta
            FROM clientes c
                WHERE c.status_conta IN ('Inativa', 'cancelada')
            ORDER BY c.nome_completo ASC;
          """).show()

+----------+--------------------+------------+
|id_cliente|       nome_completo|status_conta|
+----------+--------------------+------------+
|        24|      Agatha Moreira|     Inativa|
|       213|Alexandre Albuque...|     Inativa|
|       337|      Alexia Azevedo|     Inativa|
|       161|      Alexia Pimenta|     Inativa|
|        46|     Alícia Teixeira|     Inativa|
|        32|    Amanda Gonçalves|     Inativa|
|        65|  Ana Clara Siqueira|     Inativa|
|       131|  Ana Julia Teixeira|     Inativa|
|       230|     Ana Lívia Viana|     Inativa|
|       164|  Ana Sophia Almeida|     Inativa|
|       322|   Ana Sophia Vieira|     Inativa|
|       156|   Ana Vitória Porto|     Inativa|
|       218|       Anthony Ramos|     Inativa|
|       363|Antônio da Conceição|     Inativa|
|       372|       Arthur Campos|     Inativa|
|       386|Arthur Gabriel Ca...|     Inativa|
|       351|      Augusto Aragão|     Inativa|
|       295|       Aurora Cirino|     Inativa|
|       368| 

RANKING DE CATEGORIA GASTO POR SEGMENTO

In [144]:
spark.sql("""
    SELECT 
        h.categoria_gasto,
        
            count(h.categoria_gasto) AS Top_Categoria,
        c.segmento_cliente
    
    FROM transacoes h
    INNER JOIN clientes c 
    
        ON h.id_cliente = c.id_cliente
    
    GROUP BY h.categoria_gasto, c.segmento_cliente
    ORDER BY Top_Categoria DESC
""").show()


+---------------+-------------+----------------+
|categoria_gasto|Top_Categoria|segmento_cliente|
+---------------+-------------+----------------+
|          lazer|           34|        Standard|
|          saúde|           33|        Standard|
|         outros|           32|        Standard|
|    alimentação|           30|         Economy|
|          lazer|           28|         Premium|
|          lazer|           27|         Economy|
|       educação|           26|         Economy|
|        moradia|           26|        Standard|
|    alimentação|           26|        Standard|
|    alimentação|           23|         Premium|
|        moradia|           23|         Economy|
|     transporte|           22|        Standard|
|         outros|           22|         Economy|
|       educação|           22|        Standard|
|     transporte|           21|         Premium|
|     transporte|           20|         Economy|
|       educação|           19|         Premium|
|        moradia|   

RANKING DE CATEGORIA DE GASTO POR CLIENTES PREMIUM OU VIP

In [145]:
spark.sql("""
    SELECT 
        h.categoria_gasto,
        
            count(h.categoria_gasto) AS Top_Categoria,
        c.segmento_cliente
    
    FROM transacoes h
    INNER JOIN clientes c 
    
        ON h.id_cliente = c.id_cliente
    
    WHERE c.segmento_cliente IN ('Premium', 'VIP')
    
    GROUP BY h.categoria_gasto, c.segmento_cliente
    ORDER BY Top_Categoria DESC
""").show()


+---------------+-------------+----------------+
|categoria_gasto|Top_Categoria|segmento_cliente|
+---------------+-------------+----------------+
|          lazer|           28|         Premium|
|    alimentação|           23|         Premium|
|     transporte|           21|         Premium|
|       educação|           19|         Premium|
|        moradia|           18|         Premium|
|         outros|           17|         Premium|
|          saúde|           17|         Premium|
+---------------+-------------+----------------+



In [ ]:
spark.sql("""
    SELECT 
          id_cliente,
          nome_completo,
          FLOOR(months_between(current_date(), data_nascimento) / 12) AS idade,
          score_credito_inicial
    FROM 
         clientes
    WHERE 
          score_credito_inicial > 400.00
    ORDER BY 
          score_credito_inicial DESC
""").show()


+----------+--------------------+-----+---------------------+
|id_cliente|       nome_completo|idade|score_credito_inicial|
+----------+--------------------+-----+---------------------+
|       306|    Sra. Laura Pires|   73|               849.25|
|        44|Ana Sophia Rodrigues|   35|               847.48|
|       399|Sra. Camila Nasci...|   43|               846.42|
|       155|      Aylla Carvalho|   19|               845.74|
|       290|   Caroline Teixeira|   60|               844.02|
|       439|          Ana Vieira|   18|               843.66|
|        88|    Vinicius Fonseca|   51|               843.34|
|       246|        Diogo Araújo|   66|               842.59|
|        43|Srta. Giovanna Pires|   39|               841.92|
|        38|     Anthony Moreira|   24|               841.29|
|       368|       Aurora Moraes|   71|               840.65|
|       415|             Ayla Sá|   48|               839.73|
|        65|  Ana Clara Siqueira|   49|               836.91|
|       

ANALISES DE FRAUDE TRANSAÇÕES REIJATADAS E LOCALIZAÇÃO INCOERENTES, MULTIPLOS DISPOSITIVOS

In [158]:
spark.sql("""
    SELECT
          c.id_cliente,
          c.nome_completo, 
          t.dispositivo,
          t.status_transacao,
            collect_list(named_struct('latitude',t.latitude,'longitude', t.longitude)) as ultimas_localizacoes
            
    FROM clientes c 
    
    INNER JOIN transacoes t
        ON c.id_cliente = t.id_cliente
        
    WHERE t.status_transacao = 'rejeitada'
    GROUP BY
        c.id_cliente,
        c.nome_completo,
        t.dispositivo,
        t.status_transacao
    ORDER BY c.nome_completo ASC
          """).show(truncate=False)

+----------+-------------------------+-----------+----------------+--------------------------+
|id_cliente|nome_completo            |dispositivo|status_transacao|ultimas_localizacoes      |
+----------+-------------------------+-----------+----------------+--------------------------+
|337       |Alexia Azevedo           |celular    |rejeitada       |[{36.381904, 146.173917}] |
|106       |Ana Cecília Aparecida    |celular    |rejeitada       |[{-65.306406, 136.011601}]|
|106       |Ana Cecília Aparecida    |tablet     |rejeitada       |[{84.54971, -31.852111}]  |
|309       |Ana Lívia Teixeira       |celular    |rejeitada       |[{-55.624729, 79.705254}] |
|496       |Ana Sampaio              |celular    |rejeitada       |[{45.102336, -93.913398}] |
|164       |Ana Sophia Almeida       |celular    |rejeitada       |[{50.948265, -41.039221}] |
|205       |Ana Sophia Melo          |celular    |rejeitada       |[{76.840829, 53.042782}]  |
|76        |Ana Vitória Cavalcanti   |web        |

VERIFICAR PESSOAS QUE TIVERAM TRANSAÇÕES REJEITADAS COM SCORE ALTO ACIMA DE 500.00

In [163]:
spark.sql("""
          
        SELECT
          distinct(c.nome_completo) as Nome_Completo,
          c.id_cliente,
          t.status_transacao,
          c.score_credito_inicial,
          c.segmento_cliente
          
          
        FROM 
            clientes c
        INNER JOIN 
            transacoes t 
            
            
        ON c.id_cliente = t.id_cliente
            WHERE t.status_transacao = 'rejeitada'
        AND
            c.score_credito_inicial >= 500.00
        
        
        ORDER BY c.score_credito_inicial DESC;
          
          """).show()

+--------------------+----------+----------------+---------------------+----------------+
|       Nome_Completo|id_cliente|status_transacao|score_credito_inicial|segmento_cliente|
+--------------------+----------+----------------+---------------------+----------------+
|   Caroline Teixeira|       290|       rejeitada|               844.02|         Economy|
|    Vinicius Fonseca|        88|       rejeitada|               843.34|         Economy|
| Maria Laura Pacheco|       201|       rejeitada|               832.51|        Standard|
|        Zoe Cassiano|       345|       rejeitada|               831.89|         Economy|
|         Rhavi Costa|       189|       rejeitada|               831.58|         Economy|
| Vitor Hugo Ferreira|       318|       rejeitada|               825.99|        Standard|
|     Valentina Ramos|       302|       rejeitada|               821.85|        Standard|
|           Maya Lima|       395|       rejeitada|               815.58|         Economy|
|Dra. Aman